In [ ]:
!pip install tensorflow torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import pandas as pd
import sqlite3
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Connect to SQLite database
conn = sqlite3.connect('constitution.db')

# Load the articles DataFrame from the SQLite database
df_articles = pd.read_sql_query("SELECT * FROM articles", conn)

# Close the connection
conn.close()

# Initialize the BART model and tokenizer
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

def search_article(keyword=None, article_number=None):
    if keyword:
        result = df_articles[df_articles['article_desc'].str.contains(keyword, case=False)]
    elif article_number:
        result = df_articles[df_articles['article_id'] == article_number]
    else:
        result = pd.DataFrame()
    return result

def generate_summary(text, level):
    # Define the number of words for each level
    lengths = {1: 60, 2: 120, 3: 180, 4: 240, 5: 350}
    max_length = lengths.get(level, 120)

    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=max_length//2, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def main():
    print("Welcome to the Constitution Article Summarizer!")
    choice = input("Search by (1) Keyword or (2) Article Number? ")

    if choice == '1':
        keyword = input("Enter the keyword: ")
        articles = search_article(keyword=keyword)
    elif choice == '2':
        article_number = input("Enter the article number (e.g., 'Article 5' or 'Article 52A'): ").strip()
        articles = search_article(article_number=article_number)
    else:
        print("Invalid choice!")
        return

    if articles.empty:
        print("No articles found.")
        return

    print(f"Found {len(articles)} articles.")
    for idx, row in articles.iterrows():
        print(f"\nArticle ID: {row['article_id']}")
        print(f"Description: {row['article_desc'][:100]}...")  # Print the first 100 characters

    article_id = input("\nEnter the article ID to summarize (e.g., 'Article 5'): ").strip()
    article = articles[articles['article_id'] == article_id]

    if article.empty:
        print("Invalid article ID.")
        return

    level = int(input("Enter summary level (1-5): "))
    if level < 1 or level > 5:
        print("Invalid summary level.")
        return

    text = article.iloc[0]['article_desc']
    summary = generate_summary(text, level)
    print("\nSummary:")
    print(summary)

if __name__ == "__main__":
    main()

Welcome to the Constitution Article Summarizer!
Search by (1) Keyword or (2) Article Number? 1
Enter the keyword: Rights
Found 34 articles.

Article ID: Article 6 of Indian Constitution
Description: Rights of citizenship of certain persons who have migrated to India from Pakistan Notwithstanding an...

Article ID: Article 7 of Indian Constitution
Description: Rights of citizenship of certain migrants to Pakistan Notwithstanding anything in Articles 5 and 6, ...

Article ID: Article 8 of Indian Constitution
Description: Rights of citizenship of certain persons of India origin residing outside India Notwithstanding anyt...

Article ID: Article 10 of Indian Constitution
Description: Continuance of the rights of citizenship Every person who is or is deemed to be a citizen of India u...

Article ID: Article 11 of Indian Constitution
Description: Parliament to regulate the right of citizenship by law Nothing in the foregoing provisions of this P...

Article ID: Article 13 of Indian Constitut

In [ ]:
# Install required libraries
!pip install transformers torch pandas textstat nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00


In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    # Join words back into a single string
    processed_text = ' '.join(words)

    return processed_text

# Example usage
example_text = """
Article 5: Citizenship at the commencement of the Constitution.
At the commencement of this Constitution, every person who has his domicile in the territory of India and –
(a) who was born in the territory of India; or
(b) either of whose parents was born in the territory of India; or
(c) who has been ordinarily resident in the territory of India for not less than five years immediately preceding such commencement, shall be a citizen of India.
"""

processed_example = preprocess_text(example_text)
print(processed_example)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


article 5 citizenship commencement constitution commencement constitution every person domicile territory india – born territory india b either whose parent born territory india c ordinarily resident territory india le five year immediately preceding commencement shall citizen india


In [ ]:
import pandas as pd
import sqlite3
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
import textstat
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from transformers import pipeline
import textstat
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('constitution.db')

# Load the articles DataFrame from the SQLite database
df_articles = pd.read_sql_query("SELECT * FROM articles", conn)

# Close the connection
conn.close()

# Display the first few rows to ensure data is loaded correctly
df_articles.head()


,article_id,article_desc
0,Article 1 of Indian Constitution,"Name and territory of the Union\n(1) India, th..."
1,Article 2 of Indian Constitution,Admission or establishment of new States: Parl...
2,Article 2A of Indian Constitution,Sikkim to be associated with the Union Rep by ...
3,Article 3 of Indian Constitution,Formation of new States and alteration of area...
4,Article 4 of Indian Constitution,Laws made under Articles 2 and 3 to provide fo...


In [ ]:
# Initialize the mBART model and tokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, pipeline
model_name = "facebook/mbart-large-50-many-to-one-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [ ]:
def search_article(keyword=None, article_number=None):
    if keyword:
        result = df_articles[df_articles['article_desc'].str.contains(keyword, case=False)]
    elif article_number:
        result = df_articles[df_articles['article_id'] == article_number]
    else:
        result = pd.DataFrame()
    return result


In [ ]:
def simplify_text(text):
    return text

# Function to generate summary
def generate_summary_falc(text, level):

    f=text.split()
    max1=int(len(f)*level*0.2)
    min1=int(0.2*len(f))

    summarizer = pipeline("summarization", model="Falconsai/text_summarization")
    out=summarizer(text, max_length=max1, min_length=min1, do_sample=False)
    simplified_summary=out[0]['summary_text']

    return simplified_summary

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load the Pegasus model and tokenizer
model_name = "google/pegasus-cnn_dailymail"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

def generate_summary_peg(text, level):
    # Define the number of tokens for each level
    lengths = {1: 60, 2: 120, 3: 180, 4: 240, 5: 300}
    max_new_tokens = lengths.get(level, 120)

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=max_new_tokens, min_length=int(max_new_tokens * 0.5), num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
def main():
    print("Welcome to the Constitution Article Summarizer!")
    choice = input("Search by (1) Keyword or (2) Article Number? ")

    if choice == '1':
        keyword = input("Enter the keyword: ")
        articles = search_article(keyword=keyword)
    elif choice == '2':
        article_number = input("Enter the article number (e.g., 'Article 5' or 'Article 52A'): ").strip()
        articles = search_article(article_number=article_number)
    else:
        print("Invalid choice!")
        return

    if articles.empty:
        print("No articles found.")
        return

    print(f"Found {len(articles)} articles.")
    for idx, row in articles.iterrows():
        print(f"\nArticle ID: {row['article_id']}")
        print(f"Description: {row['article_desc'][:100]}...")  # Print the first 100 characters

    article_id = input("\nEnter the article ID to summarize (e.g., 'Article 5'): ").strip()
    article = articles[articles['article_id'] == article_id]

    if article.empty:
        print("Invalid article ID.")
        return

    level = int(input("Enter summary level (1-5): "))
    if level < 1 or level > 5:
        print("Invalid summary level.")
        return

    text = article.iloc[0]['article_desc']
    global sum1
    global sum2
    summary = generate_summary_falc(text, level)
    sum1=summary
    print("\nSummary from Extractive Model:")
    print(summary)

    summary = generate_summary_peg(text, level)
    sum2=summary
    print("\nSummary from Abstractive Model:")
    print(summary)


if __name__ == "__main__":
    main()


Welcome to the Constitution Article Summarizer!
Search by (1) Keyword or (2) Article Number? 1
Enter the keyword: Rights
Found 34 articles.

Article ID: Article 6 of Indian Constitution
Description: Rights of citizenship of certain persons who have migrated to India from Pakistan Notwithstanding an...

Article ID: Article 7 of Indian Constitution
Description: Rights of citizenship of certain migrants to Pakistan Notwithstanding anything in Articles 5 and 6, ...

Article ID: Article 8 of Indian Constitution
Description: Rights of citizenship of certain persons of India origin residing outside India Notwithstanding anyt...

Article ID: Article 10 of Indian Constitution
Description: Continuance of the rights of citizenship Every person who is or is deemed to be a citizen of India u...

Article ID: Article 11 of Indian Constitution
Description: Parliament to regulate the right of citizenship by law Nothing in the foregoing provisions of this P...

Article ID: Article 13 of Indian Constitut

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]


Summary from Extractive Model:
Provisions as to Comptroller and Auditor General of India The Auditor general of India holding office immediately before the commencement of this Constitution shall, unless he has elected otherwise, be entitled to such salaries and rights in respect of leave of absence and pension as are provided for under clause 3 of Article 148

Summary from Abstractive Model:
The Auditor General of India holding office immediately before the commencement of this Constitution shall, unless he has elected otherwise, become on such commencement the Comptroller and Auditor General of India.<n>He is entitled to such salaries and to such rights in respect of leave of absence and pension as are provided for under clause ( 3 ) of Article 148 in respect of the Comptroller and Auditor General of India and be entitled to continue to hold office until the expiration of his term of office.


In [ ]:
new=sum2.split("<n>")
for sentence in new:
  print(sentence)

The Auditor General of India holding office immediately before the commencement of this Constitution shall, unless he has elected otherwise, become on such commencement the Comptroller and Auditor General of India.
He is entitled to such salaries and to such rights in respect of leave of absence and pension as are provided for under clause ( 3 ) of Article 148 in respect of the Comptroller and Auditor General of India and be entitled to continue to hold office until the expiration of his term of office.


In [ ]:
!pip install googletrans==4.0.0-rc1



  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=dbc01905c103952d562c9c6b869cafbf96fd63e7ad898a73e831459232d5bdb0
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [ ]:
from googletrans import Translator

# Initialize the translator
translator = Translator()

def translate(text: str, dest_language: str) -> str:
    # Translate the text
    translation = translator.translate(text, dest=dest_language)
    return translation.text

# Example usage
text =sum2

translated_text_hi = translate(text, 'hi')  # Hindi
translated_text_kn = translate(text, 'kn')  # Kannada
translated_text_te = translate(text, 'te')  # Telugu

print("Translated to Hindi:", translated_text_hi)
print("Translated to Kannada:", translated_text_kn)
print("Translated to Telugu:", translated_text_te)


Translated to Hindi: इस संविधान के शुरू होने से तुरंत पहले भारत के ऑडिटर जनरल होल्डिंग ऑफिस, जब तक कि वह अन्यथा नहीं चुने गए, तब तक इस तरह के कमेंट्रोलर और ऑडिटर जनरल ऑफ इंडिया।अनुपस्थिति और पेंशन की छुट्टी के रूप में भारत के नियंत्रक और ऑडिटर जनरल के संबंध में अनुच्छेद 148 के खंड (3) के तहत प्रदान किया जाता है और कार्यालय के कार्यकाल की समाप्ति तक पदभार संभालने के हकदार हैं।
Translated to Kannada: ಈ ಸಂವಿಧಾನದ ಪ್ರಾರಂಭದ ಮೊದಲು ಭಾರತದ ಲೆಕ್ಕಪರಿಶೋಧಕ ಜನರಲ್, ಅವರು ಬೇರೆ ರೀತಿಯಲ್ಲಿ ಆಯ್ಕೆಯಾಗದಿದ್ದರೆ, ಅಂತಹ ಪ್ರಾರಂಭದಲ್ಲಿ ಭಾರತದ ಕಂಟ್ರೋಲರ್ ಮತ್ತು ಆಡಿಟರ್ ಜನರಲ್ ಆಗುತ್ತಾರೆ. <n> ಅಂತಹ ಸಂಬಳ ಮತ್ತು ಅಂತಹ ಹಕ್ಕುಗಳಿಗೆ ಅವರು ಅರ್ಹರಾಗಿದ್ದಾರೆಕಂಟ್ರೋಲರ್ ಮತ್ತು ಭಾರತದ ಲೆಕ್ಕಪರಿಶೋಧಕ ಜನರಲ್ಗೆ ಸಂಬಂಧಿಸಿದಂತೆ ಆರ್ಟಿಕಲ್ 148 ರ ಷರತ್ತು (3) ರ ಅಡಿಯಲ್ಲಿ ಒದಗಿಸಲಾದ ಅನುಪಸ್ಥಿತಿ ಮತ್ತು ಪಿಂಚಣಿ ರಜೆ ಮತ್ತು ಅವರ ಕಚೇರಿ ಅವಧಿ ಮುಗಿಯುವವರೆಗೂ ಅಧಿಕಾರ ವಹಿಸಿಕೊಳ್ಳುವುದನ್ನು ಮುಂದುವರಿಸಲು ಅರ್ಹರಾಗಿರಿ.
Translated to Telugu: ఈ రాజ్యాంగం ప్రారంభానికి ముందే భారత ఆడిటర్ జనరల్ ఆఫ్ ఇండియా హోల్డింగ్ ఆఫీస్కంప్ట్రోలర్ మరియు ఆడిటర్ జనరల్ ఆఫ్ ఇండియాకు సంబంధించి ఆర్టికల్ 148 లోని నిబంధన 